ELEC-E5550 - Statistical Natural Language Processing
# SET 5: Machine Translation Evaluation

# Released: 07.03.2023
# Deadline: 20.03.2023 at midnight

In this assignment, you will learn how machine translation systems can be automatically evaluated, using the BLEU score.

KEYWORDS:
* BLEU

Do not use external libraries in this assignment. Python standard library is allowed.

# Table of contents

* [TASK 1: BLEU score](#task1)
    * [Step 1.1: Match N-grams](#subtask1_1)
    * [Step 1.2: Match lengths](#subtask1_2)
    * [Step 1.3: Put it together](#subtask1_3)
    * [Step 1.4: Reflect on BLEU](#subtask1_4)

Evaluation is a particularly difficult topic in machine translation. Human evaluations are preferred, but expensive. There are multiple automatic evaluation metrics in use. The most common one is BLEU. In this assignment you will implement BLEU calculation - it is computationally very simple.

## TASK 1 <a class="anchor" id="task1"></a>
## BLEU score

**BLEU**, or bilingual evaluation understudy, was proposed in 2002, here: https://www.aclweb.org/anthology/P02-1040.pdf. Take read through the paper, it is clearly written and provides not just the algorithm, but also context and reasons for the design choices. We will implement the algorithm as described by the paper, so you can refer to it.

### High-level overview

The essential requirement for BLEU is an annotated test corpus. The test corpus consists of source language segments and multiple (or at least one) reference translations for each segment. The reference translations are made by human translators. Multiple reference translations are used to account for the fact that there is no single correct translation.

A machine translation system produces a hypothesis translation for each source segment. The hypothesis is then compared to all the references. BLEU looks for matching N-grams of different lengths N. The more matches, the better. Additionally, BLEU compares the length of the hypothesis with the lengths of the translations - brevity is penalized. These two components are combined into a score between 0 and 1.

## 1.1 Match N-grams (3 Points) <a class="anchor" id="subtask1_1"></a>
Specifically, BLEU uses a concept called modified precision. The hypothesis n-grams can find a match in any reference - but if the same n-gram occurs multiple times in the hypothesis, the number of matches is clipped to the maximum number of times the n-gram occurs in any single reference. The intuition is simple - the references are likely to have many words and phrases in common. Without the clipping, repeating the same likely words in the hypothesis would yield an inflated BLEU score. The hypothesis "the the the the the the the" would find many matches from almost any set of references.

In [1]:
"""
You may use this ngram forming implementation in your solution.
This returns a generator. If you absolutely need a list, you can
simply call
>>> list(form_ngrams(...))
"""
from collections import deque

def form_ngrams(tokens, n):
    """Forms all ngrams from tokens
    
    Arguments
    ---------
    tokens : iterable
        Tokens to form n-grams from
    n : int
        The length of ngrams to form
    
    Yields
    ------
    tuple
        Yields each ngram as a tuple of tokens.   
    """
    window = deque(maxlen=n)
    for i, token in enumerate(tokens, start=1):
        window.append(token)
        if i >= n:
            yield tuple(window)
            
print(list(form_ngrams(["furiously", "sleep", "ideas", "green", "colorless"], 3)))

[('furiously', 'sleep', 'ideas'), ('sleep', 'ideas', 'green'), ('ideas', 'green', 'colorless')]


In [2]:
from collections import Counter

def modified_precision(hypothesis, references, n):
    """Matches hypothesis N-grams to references at given length N
    
    Hypothesis and references are given as plain token sequences. This function
    forms N-grams of appropriate lengths.
    
    Count the number of times each N-gram occurs in the hypothesis.
    Then take the union of N-gram counts in all the references:
    for each N-gram, keep track of the maximum number of times it occurs in a single
    reference. This forms the reference counts.
    Finally, count the clipped hits - sum each N-gram count in the hypothesis,
    but clip those hypothesis counts to the reference counts.
    
    This returns the hits and number of candidates separately, since those values
    are aggregated over the full corpus in BLEU.
    The modified precision for this single segment would be clipped_hits / num_candidates
    
    Arguments
    ---------
    hypothesis : list of str
        List of tokens - the hypothesis translation.
        E.G. ["alo", "mundo"]
    references : list of lists of str
        List of references - and each reference is a list of tokens.
        E.G. [["hola", "mundo"], ["alo", "tierra"]]
    
    Returns
    -------
    int
        The number of candidate N-grams of length N in the hypothesis that found
        a match.
    int
        The total number of candidate N-grams of length N in the hypothesis.
    """
    print('Hypothesis')
    print(hypothesis)
    print('References')
    print(references)
    print(n)
    print(list(form_ngrams(hypothesis,n)))
    hyp_counts = Counter(form_ngrams(hypothesis,n))
    
    num_candidates = sum(hyp_counts.values())
    if num_candidates == 0:
        # If hypothesis is shorter than N, we get here.
        # We return 0, 1 instead of 0, 0, so that we don't
        # run into divide-by-zero errors.
        return 0, 1
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    lista=[]
    for ref in references:
        ref_ngrams=list(form_ngrams(ref,n))
        lista.append(ref_ngrams)

    #lista2=[]
    #for elem in lista:
    #    for l in elem:
    #        lista2.append(l)
    #lista=lista2
    clipped_hits=0
    lista2=[]
    
    for l in lista:
        dic=Counter(l)
        lista2.append(dic)
    
    lista=lista2
    print(lista)
    
    for k in hyp_counts:
        v=hyp_counts[k]
        n=0
        maxnum=0
        for d in lista:
            n=0
            for k2 in d:
                if(k2==k):
                    n=d[k2]        
            if(n>maxnum):
                maxnum=n
        if(maxnum<v):
            v=maxnum
        clipped_hits=clipped_hits+v
        
    # Return the numerator and denominator separately, since
    # they get aggregated over the whole corpus.
    print(clipped_hits)
    return clipped_hits, num_candidates

    

In [3]:
from nose.tools import assert_equal

# Basic sanity check:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 1), (4, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 2), (3, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 3), (2, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 4), (1, 1))

# Small difference:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 1), (3, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 2), (2, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 3), (1, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 4), (0, 1))

# More references:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 1), (4, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 2), (3, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 3), (1, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 4), (0, 1))

# Clipping:
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 1), (2, 4))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 2), (1, 3))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 3), (0, 2))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 4), (0, 1))

# With real text:
assert_equal(modified_precision(["waters", "destroyed", "perfume"], 
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                1), (2, 3))
assert_equal(modified_precision(["waters", "destroyed", "perfume"], 
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                2), (0, 2))

austen_hypo = ["it", "is", "a", "widely", "accepted", "truth", 
              "that", "an", "unmarried", "wealthy", "man", 
              "necessarily", "needs", "a", "wife", "alongside", "him"]
austen_ref = [["it", "is", "a", "truth", "universally", "acknowledged",
               "that", "a", "single", "man", "in", "posession", "of","a""good","fortune",
               "must", "be", "in", "want", "of", "a", "wife"],
              ["everybody", "knows", "the","rich","man","necessarily", "needs", "a", "wife"]]
assert_equal(modified_precision(austen_hypo, austen_ref, 3), (4,15))
assert_equal(modified_precision(austen_hypo, austen_ref, 4), (2,14))
assert_equal(modified_precision(austen_hypo, austen_ref, 5), (1,13))
assert_equal(modified_precision(austen_hypo, austen_ref, 6), (0,12))

assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                1), (2, 3))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                2), (0, 2))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                3), (0, 1))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                4), (0, 1))

assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 1), (2, 7))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 2), (0, 6))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 3), (0, 5))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 4), (0, 4))



Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'd']]
1
[('a',), ('b',), ('c',), ('d',)]
[Counter({('a',): 1, ('b',): 1, ('c',): 1, ('d',): 1})]
4
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'd']]
2
[('a', 'b'), ('b', 'c'), ('c', 'd')]
[Counter({('a', 'b'): 1, ('b', 'c'): 1, ('c', 'd'): 1})]
3
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'd']]
3
[('a', 'b', 'c'), ('b', 'c', 'd')]
[Counter({('a', 'b', 'c'): 1, ('b', 'c', 'd'): 1})]
2
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'd']]
4
[('a', 'b', 'c', 'd')]
[Counter({('a', 'b', 'c', 'd'): 1})]
1
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'e']]
1
[('a',), ('b',), ('c',), ('d',)]
[Counter({('a',): 1, ('b',): 1, ('c',): 1, ('e',): 1})]
3
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'e']]
2
[('a', 'b'), ('b', 'c'), ('c', 'd')]
[Counter({('a', 'b'): 1, ('b', 'c'): 1, ('c', 'e'): 1})]
2
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c', 'e']]
3

## 1.2 Match lengths (1 Point) <a class="anchor" id="subtask1_2"></a>

BLEU penalizes short hypotheses. This counter part is needed for precision, because precision by itself encourages adding only the most certain candidates. Typically recall is used as precision's pair, why not here? You can read the original paper to see. Instead of recall, BLEU uses a Brevity Penalty, which is computed over the whole test corpus. Here we just find the values which are aggregated - the hypothesis length and the closest reference length.

In [4]:
def brevity_penalty_values(hypothesis, references):
    """Computes the length of the hypothesis and find the closest reference length.
    
    Pick the shortest reference length if there are many equally close
    
    Arguments
    ---------
    hypothesis : list of str
        List of tokens - the hypothesis translation.
        E.G. ["alo", "mundo"]
    references : list of lists of str
        List of references - and each reference is a list of tokens.
        E.G. [["hola", "mundo"], ["alo", "tierra"]]
        
    Returns
    -------
    int
        The length of the hypothesis
    int
        The length of reference, which is closest to the hypothesis in length
        
    Note
    ----
    This just considers length in tokens - this doesn't consider N-grams.
    """
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    
    hyplen=len(hypothesis)
    closest_reflen=2000000
    prevdist=100000
    for ref in references:
        leng=len(ref)
        dist=abs(leng-hyplen)
        if(dist<prevdist):
            prevdist=dist
            closest_reflen=leng
        elif(dist==prevdist and leng<closest_reflen):
            closest_reflen=leng
            
        
 
    
    return hyplen, closest_reflen

In [5]:
from nose.tools import assert_equal

# Basic cases:
assert_equal(brevity_penalty_values(["a","b","c"], [["e","f","g"],["a","b"]]), (3,3))
assert_equal(brevity_penalty_values(["a","b","c"], [["e","f","g","h"],["a","b"]]), (3,2))
assert_equal(brevity_penalty_values(["a","b","c","d"], [["e","f","g","h","i","j"],["a","b"]]), (4,2))
assert_equal(brevity_penalty_values(["a","b","c","d"], [["e","f","g","h"],["a","b"]]), (4,4))
assert_equal(brevity_penalty_values(["a","b"], [["e","f","g","h"],["a","b","c","d","e"]]), (2,4))
assert_equal(brevity_penalty_values([], [["a","b","c","d","e"],[]]), (0,0))
assert_equal(brevity_penalty_values(["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q"],
                                    [["r","s","t","u","v","x","y"],
                                     ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v"]]), 
             (17, 22))




## 1.3 Put it together <a class="anchor" id="subtask1_3"></a>

The key in BLEU is to aggregate the statistics from above over a large test corpus. BLEU computed on an individual sentence does not correlate well with human scores. Even if this sentence-wise BLEU is computed for all individual sentences in a test corpus and then averaged, the resulting metric is much less useful than when a single BLEU score is computed over the full corpus.

The modified precision aggregation happens at each N-gram length separately. Typically, BLEU-4 is computed, meaning maximum N-gram length 4. In that case, 8 precision values are collected: total number of hits and total number of candidates at each each N.

The length values just consider length in tokens, so only two values are needed: the total length of the hypotheses and the total length of the best matching reference lengths. The brevity penalty (BP) is then computed as follows:
$$
BP = 1,~c>r
$$
$$
BP = exp(1-\dfrac{r}{c}), c\leq r
$$
with aggregated hypothesis length $c$ and aggregated closest reference length $r$.

The metric is finally computed by:
$$
BLEU = BP\times exp\left( \sum_n^N \left[\dfrac{1}{N}log(p_n)\right]\right)
$$

Here aggregated modified precision at N-gram length $n$ is simply computed as: $p_n = \dfrac{\text{Clipped Hits}}{\text{Total Candidates}} $


In [13]:
# Use these exp and log operations:
from math import exp, log
EPSILON = 1e-10  # This is used for smoothing zero counts (already implemented)

def BLEU(hypothesis_set, references_set, max_n=4):
    """Computes BLEU over a corpus
    
    Use your two earlier functions.
    """

    # Collect these statistics over the corpus ...
    hits_totals = {N: 0 for N in range(1,max_n+1)} # Hits by N-gram N
    num_candidates_totals = {N: 0 for N in range(1,max_n+1)} # Num candidates by N
    hyplen_total = 0
    closest_reflen_total = 0
    
    
  
  
    
    # ... in the following loop:
    for hypothesis, references in zip(hypothesis_set, references_set):
        for i in range(1,max_n+1):
            ret=modified_precision(hypothesis, references, i)
            hits=ret[0]
            num_candidates=ret[1]
            hits_totals[i]=hits+hits_totals[i]
            num_candidates_totals[i]=num_candidates+num_candidates_totals[i]
            
    #print(list_hits)
    #print(list_candidates)
    # Additionally in BLEU, substituting a low value for zero counts is used
    # to avoid log(0) problems.
    # Smooth zero counts:
    hits_totals = {N: hits if hits > 0 else EPSILON for N, hits in hits_totals.items()}
    
    # Now, compute brevity penalty based on 
    # hyplen_total and closest_reflen_totals
    # Corner case: if hyplen_total == 0: brevity_penalty = 0.
    # YOUR CODE HERE
   
    
    
    for hypothesis, references in zip(hypothesis_set, references_set):
        ret2=brevity_penalty_values(hypothesis, references);
        hyplen_total=hyplen_total+ret2[0]
        closest_reflen_total=+closest_reflen_total+ret2[1]
        
    c=hyplen_total
    r=closest_reflen_total
    
    if(c==0):
        bp=0
    elif(c>r):
        bp=1
    else:
        bp=exp(1-(r/c))
        
   
    print(bp)
    
    
    
    #raise NotImplementedError()

    # Finally compute the precisions, and weight them uniformly by 1 / max_n
    # Then multiply by brevity penalty and return the final score.
    # YOUR CODE HERE
    value=0
    for i in range(1,max_n+1):
        hits=hits_totals[i]
        num_candidates=num_candidates_totals[i]
        pn=hits/num_candidates
        value=log(pn)+value
    value=value/max_n
    value=exp(value)
    final_score=bp*value
    return final_score
    #raise NotImplementedError()

In [16]:
hyps =[
    ["it"," cannot"," into ","a"," basis"," for"," the ","european"," constitution"]
]

hyps2 =[
    ["she", "can ","be"," used"," as ","a ","basis"," for"," the"," installation"," of"," a"," european"," constitution"]
]

ref=[
    ["it", "can", "serve", "as", "a" ,"basis"," for"," the ","establishment"," of ","a", "european" ,"constitution"]
]

print('ciao')
b1=BLEU(hyps1, ref)
b2=BLEU(hyps2, ref)
print(b1)
print(b2)

ciao
Hypothesis
['it', ' cannot', ' into ', 'a', ' basis', ' for', ' the ', 'european', ' constitution']
References
['it', 'can', 'serve', 'as', 'a', 'basis', ' for', ' the ', 'establishment', ' of ', 'a', 'european', 'constitution']
1
[('it',), (' cannot',), (' into ',), ('a',), (' basis',), (' for',), (' the ',), ('european',), (' constitution',)]
[Counter({('i',): 1, ('t',): 1}), Counter({('c',): 1, ('a',): 1, ('n',): 1}), Counter({('e',): 2, ('s',): 1, ('r',): 1, ('v',): 1}), Counter({('a',): 1, ('s',): 1}), Counter({('a',): 1}), Counter({('s',): 2, ('b',): 1, ('a',): 1, ('i',): 1}), Counter({(' ',): 1, ('f',): 1, ('o',): 1, ('r',): 1}), Counter({(' ',): 2, ('t',): 1, ('h',): 1, ('e',): 1}), Counter({('e',): 2, ('s',): 2, ('t',): 2, ('a',): 1, ('b',): 1, ('l',): 1, ('i',): 1, ('h',): 1, ('m',): 1, ('n',): 1}), Counter({(' ',): 2, ('o',): 1, ('f',): 1}), Counter({('a',): 1}), Counter({('e',): 2, ('u',): 1, ('r',): 1, ('o',): 1, ('p',): 1, ('a',): 1, ('n',): 1}), Counter({('t',): 3, 

In [8]:
from numpy.testing import assert_almost_equal
from nose.tools import assert_equal

# Basic corpus input:
hyps = [["a", "b", "c","d"],
        ["colourless", "green", "ideas"],
       ["a","c","d"]]        
references = [[["a","b","c"], ["a","b","c","d","e"]], 
              [["ideas", "green", "colourless", "sleep"],["colourless", "green"]],
              [["a","b","c"], ["a","b","c","d","e"]]]
assert_almost_equal(BLEU(hyps, references), 0.5873949094699213, 2)
# More corpus input:
hyps = [["waters", "destroyed", "perfume"],
        ["the", "the", "the", "the", "the", "the", "the"],
        ["it", "is", "a", "widely", "accepted", "truth", 
         "that", "an", "unmarried", "wealthy", "man", 
         "necessarily", "needs", "a", "wife", "alongside", "him"]]        
references = [[["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
              [["the", "cat", "is", "on", "the", "mat"], ["there", "is", "a", "cat", "on", "the", "mat"]],
              [["it","is","a","truth","universally","acknowledged",
                "that", "a", "single", "man", "in", "posession", "of","a""good","fortune",
               "must","be","in","want","of","a","wife"],
               ["everybody", "knows", "the","rich","man","necessarily", "needs", "a", "wife"]]]
assert_almost_equal(BLEU(hyps, references), 0.1502348037292212, 2)
# Exactly the same:
assert_equal(BLEU([[1,2,3,4]], [[[1,2,3,4]]]), 1.0)
# Slightly different:
assert_almost_equal(BLEU([[5,1,2,3,4]], [[[1,2,3,4,5]]]), 0.7071067811865475, 2)
# No hypothesis:
assert_equal(BLEU([[]], [[["Silence"]]]), 0.)
# No hits at N=4:
assert_almost_equal(BLEU([[1,2,3,4]], [[[1,2,3,5]]]), 0.0022360679774997894, 2)





Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c'], ['a', 'b', 'c', 'd', 'e']]
1
[('a',), ('b',), ('c',), ('d',)]
[Counter({('a',): 1, ('b',): 1, ('c',): 1}), Counter({('a',): 1, ('b',): 1, ('c',): 1, ('d',): 1, ('e',): 1})]
4
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c'], ['a', 'b', 'c', 'd', 'e']]
2
[('a', 'b'), ('b', 'c'), ('c', 'd')]
[Counter({('a', 'b'): 1, ('b', 'c'): 1}), Counter({('a', 'b'): 1, ('b', 'c'): 1, ('c', 'd'): 1, ('d', 'e'): 1})]
3
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c'], ['a', 'b', 'c', 'd', 'e']]
3
[('a', 'b', 'c'), ('b', 'c', 'd')]
[Counter({('a', 'b', 'c'): 1}), Counter({('a', 'b', 'c'): 1, ('b', 'c', 'd'): 1, ('c', 'd', 'e'): 1})]
2
Hypothesis
['a', 'b', 'c', 'd']
References
[['a', 'b', 'c'], ['a', 'b', 'c', 'd', 'e']]
4
[('a', 'b', 'c', 'd')]
[Counter(), Counter({('a', 'b', 'c', 'd'): 1, ('b', 'c', 'd', 'e'): 1})]
1
Hypothesis
['colourless', 'green', 'ideas']
References
[['ideas', 'green', 'colourless', 'sleep'], ['col

## 1.4 Reflect on BLEU <a class="anchor" id="subtask1_4"></a>
Briefly answer the following questions:

1. Will providing more references increase the BLEU score? Why?

2. Why is the brevity penalty is used instead of the more traditional recall?

3. Will a human translator always get a BLEU score of 1? Why?

4. Autograding these written questions is impossible. Or is it? How could we use BLEU to autograde these answers? The upside would be to save a lot of teaching assistant time, but what would be some downsides?

1. Yes. More references increase for sure the blue score since more references means more matching ngrams (or in the worst case  zero matching). We have also this statement in the research paper.

2. From the research paper: We have two sentences 'I always invariably perpetually do.' and 'I always do.' Between these sentences using the recall we evaluate as the best translation the first one since it recalls all the three possible translation of the word; however the best translation is the one in which only one alternative is picked. For this reason recall is not the best metric to pair with precision and it' better to use brevity penalty.

3. Not necessarily in fact most of the time there not exists a unique translation for a sentence. So for this reason if a human translates a sentence in a different way from the references it doesn't get score 1.  

4. We should have as references a possible set of sentences that represent possible right answers. For example: the first and third question are yes or no questions so if there is a matching in the answer with yes or no between candidate and references we should grade partially the answer for first and third question. Autograding why questions is more difficult but we could provide a lot of references that can in some way reflect what we are expecting from the answers.
